# Harmonization of MIDRC Imaging Study Descriptions using the LOINC Playbook (for Sequestered Data)
---
by Chris Meyer, PhD (edited by Johnbright Anyaibe, M.S for validatestaging environment)

Manager of Data and User Services at the Center for Translational Data Science at University of Chicago

November 2022

---
This Jupyter notebook demonstrates how to map the imaging_study node's `study_description` and `modality` properties to a `loinc_code` by utilizing the MIDRC DQH committee's [LOINC mapping table in GitHub](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out). Once the imaging study has been mapped to a LOINC code, the code can be used to derive other LOINC properties. The following [LOINC properties](https://github.com/uc-cdis/midrc_dictionary/blob/deae581f0fb8b9ae5add1458d7882e189ba97af6/gdcdictionary/schemas/imaging_study.yaml#L77) are on the `imaging_study` node of the [MIDRC data dictionary](https://data.midrc.org/dd):
* loinc_code
* loinc_long_common_name
* loinc_method
* loinc_system
* loinc_contrast

In [1]:
# Import Python Packages and scripts
import pandas as pd
import numpy as np
from pathlib import Path
import sys, os, copy, datetime, shutil


import gen3
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query


In [2]:
# download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
loinc_dir = "/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC" #set your working dir
os.chdir(loinc_dir)
os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py -O expansion.py")
from expansion import Gen3Expansion


--2023-05-19 10:33:43--  https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211548 (207K) [text/plain]
Saving to: 'expansion.py'

     0K .......... .......... .......... .......... .......... 24% 1.32M 0s
    50K .......... .......... .......... .......... .......... 48% 2.71M 0s
   100K .......... .......... .......... .......... .......... 72% 7.11M 0s
   150K .......... .......... .......... .......... .......... 96% 11.4M 0s
   200K ......                                                100%  132K=0.07s

2023-05-19 10:33:43 (3.05 MB/s) - 'expansion.py' saved [211548/211548]

/Users/johnbrightanyaaibe/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__

In [3]:
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://validatestaging.midrc.org/identity
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
vapi = 'https://validatestaging.midrc.org'
vcred = '/Users/johnbrightanyaaibe/Downloads/midrc-validatestaging-credentials.json'
vauth = Gen3Auth(vapi, refresh_file=vcred) # authentication class
vsub = Gen3Submission(vapi, vauth) # submission class
vquery = Gen3Query(vauth) # query class
vexp = Gen3Expansion(vapi,vauth,vsub) # class with some custom scripts
vexp.get_project_ids()


Getting all project_ids you have access to in the data commons.
['SEQ_NCU-A4', 'SEQ_NCU-R4', 'SEQ_Open-A3', 'SEQ_Open-R3', 'SEQ_Restricted-DT']


['SEQ_NCU-A4', 'SEQ_NCU-R4', 'SEQ_Open-A3', 'SEQ_Open-R3', 'SEQ_Restricted-DT']

## Prepare the mapping and filter tables
---
Get the mapping table and the filtering attributes tables from the [MIDRC GitHub repository](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out) and reformat them to prepare for the LOINC mapping.

Prior to this, you will need to clone the repo using `git clone git@github.com:MIDRC/midrc_dicom_harmonization.git`


In [4]:
## Set the directory to your copy of the GitHub repo and pull main branch to pull latest updates from GitHub
git_dir = "/Users/johnbrightanyaaibe/Documents/GitHub/MIDRC/midrc_dicom_harmonization/"
os.chdir(git_dir)
os.system("git checkout main")
os.system("git pull origin main")
os.chdir(loinc_dir)


Already on 'main'


Your branch is up to date with 'origin/main'.
Already up to date.


From github.com:MIDRC/midrc_dicom_harmonization
 * branch            main       -> FETCH_HEAD


In [5]:
## Create the output/working directory
now = datetime.datetime.now()
today = "{}-{}-{}".format(now.year, now.month, now.day)

results_dir = "{}/results_{}".format(loinc_dir,today)
Path(results_dir).mkdir(parents=True, exist_ok=True)

In [6]:
mapping_file = "{}/out/StudyDescription_mapping_table.csv".format(git_dir)
mapping = pd.read_csv(mapping_file,dtype=str) #['Modality', 'StudyDescription', 'LOINC code', 'L-Long Common Name']

shutil.copy2(mapping_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

mapping.rename(columns={"StudyDescription":"study_description","Modality":"study_modality","LOINC code":"loinc_code","L-Long Common Name":"loinc_long_common_name"},inplace=True)
mapping.drop_duplicates(inplace=True)
mapping['study_description'] = mapping.apply(lambda row: row['study_description'].casefold(),axis=1)
mapping['study_modality'] = mapping.apply(lambda row: row['study_modality'].casefold(),axis=1)

## Change any "(blank)" values for study_description to "[blank]" to match the mapping table
mapping.replace({"study_description":{
    '(blank)':'[blank]'}},
    inplace=True)

## remove any leading/trailing whitespaces in the codes/descriptions
mapping['loinc_code'] = mapping['loinc_code'].str.strip()
mapping['loinc_long_common_name'] = mapping['loinc_long_common_name'].str.strip()
mapping['study_description'] = mapping['study_description'].str.strip()
mapping['study_modality'] = mapping['study_modality'].str.strip()

## Summarize the mapping table
modalities = list(set(mapping['study_modality']))
descriptions = list(set(mapping['study_description']))
mapping_codes = list(set(mapping['loinc_code']))
mapping_names = list(set(mapping['loinc_long_common_name']))

display(mapping)
print("study_modality values in mapping table: {}".format(modalities))
print("Number of unique LOINC codes in mapping table: {}".format(len(mapping_codes)))
print("Number of unique LOINC Long Common Names in mapping table: {}".format(len(mapping_names)))


,study_modality,study_description,loinc_code,loinc_long_common_name
0,ct,[blank],25045-6,CT Unspecified body region
1,ct,pet ct fdg imag skull to thigh,81555-5,PET+CT Guidance for localization of tumor of W...
2,ct,chest pe(adult),79077-4,CTA Pulmonary arteries for pulmonary embolus W...
3,ct,ct chest pulmonary angio with iv con,79077-4,CTA Pulmonary arteries for pulmonary embolus W...
4,ct,ct chest pulmonary embolism (ctpe),79077-4,CTA Pulmonary arteries for pulmonary embolus W...
...,...,...,...,...
301,mg,[blank],36625-2,MG Breast Views
302,nm,[blank],49118-3,NM Unspecified body region Views
303,st,[blank],43526-3,SPECT Unspecified body region
304,[blank],bronchoscopy,18744-3,Bronchoscopy study


study_modality values in mapping table: ['pt, ctpt', 'mr', 'ct', 'cr, dx', 'rf', 'us', 'nm', 'mg', 'st', '[blank]']
Number of unique LOINC codes in mapping table: 83
Number of unique LOINC Long Common Names in mapping table: 83


In [7]:
filters_file = "{}/out/StudyDescription_filtering_attributes.csv".format(git_dir)
filters = pd.read_csv(filters_file,dtype=str) #['LOINC code', 'L-Long Common Name', 'L-Method', 'L-System', 'Rad.Timing']

shutil.copy2(filters_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

filters.rename(columns= {
    'LOINC code':'loinc_code',
    'L-Long Common Name':'loinc_long_common_name',
    'L-Method':'loinc_method',
    'Rad.Timing':'loinc_contrast',
    'MIDRC-System':'loinc_system'
    },
    inplace=True,
    errors='ignore'
)
filters.drop(columns='L-System',inplace=True,errors='ignore')

## remove any leading/trailing whitespaces
filters['loinc_code'] = filters['loinc_code'].str.strip()
filters['loinc_long_common_name'] = filters['loinc_long_common_name'].str.strip()

filter_codes = list(set(filters['loinc_code']))
filter_names = list(set(filters['loinc_long_common_name']))
print("Number of unique LOINC codes in filters table: {}".format(len(filter_codes)))
print("Number of unique LOINC Long Common Names in mapping table: {}".format(len(filter_names)))


missing_codes = set(mapping_codes).difference(set(filter_codes))
missing_names = set(mapping_names).difference(set(filter_names))

print("Number of LOINC codes in mapping table missing from filters table: {}\n\t{}".format(len(missing_codes),missing_codes))
print("Number of LOINC names in mapping table missing from filters table: {}\n\t{}".format(len(missing_names),missing_names))

display(filters)


Number of unique LOINC codes in filters table: 82
Number of unique LOINC Long Common Names in mapping table: 82
Number of LOINC codes in mapping table missing from filters table: 1
	{'18744-3'}
Number of LOINC names in mapping table missing from filters table: 1
	{'Bronchoscopy study'}


,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,35889-5,RF Guidance for bronchoscopy of Chest,RF,NaN,Chest
1,36813-4,CT Abdomen and Pelvis W contrast IV,CT,W,Abdomen
2,42274-1,CT Abdomen and Pelvis WO and W contrast IV,CT,WO & W,Abdomen+Pelvis
3,36952-0,CT Abdomen and Pelvis WO contrast,CT,WO,Abdomen+Pelvis
4,79103-8,CT Abdomen W contrast IV,CT,W,Abdomen
...,...,...,...,...,...
77,83017-4,XR Chest View and Abdomen Supine and Upright,XR,NaN,Chest && Abdomen
78,30745-4,XR Chest Views,XR,NaN,Chest
79,24899-7,XR Ribs Views,XR,NaN,Chest>Ribs
80,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified


## Get the imaging_study information from MIDRC Staging
---
* Use the [`Gen3Expansion.get_node_tsvs()` SDK function](https://github.com/cgmeyer/gen3sdk-python/blob/389e3945482439ace6e4536e6d0e35c6e48de9c9/expansion/expansion.py#L219) to get all the imaging_studies in MIDRC Staging (staging.midrc.org).

* The function `exp.get_node_tsvs()` will return a master dataframe containing all the imaging_study data from the projects specified.

### In this version of the notebook, we're only getting three projects: 
* SEQ_Open-R3 (data from RSNA)
* SEQ_Open-A3 (data from ACR)
* SEQ_NCU-R4 (NCU data from RSNA)
* SEQ_NCU-A4 (NCU data from ACR)


In [8]:
os.chdir(loinc_dir)
projects = ['SEQ_Open-R3','SEQ_Open-A3','SEQ_NCU-R4', 'SEQ_NCU-A4' ]
#vt = vexp.get_node_tsvs(node='imaging_study', overwrite=False, projects=projects,outdir=results_dir)
vt = vexp.get_node_tsvs(node='imaging_study', overwrite=True, projects=projects,outdir=results_dir)



Output written to file: /Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/imaging_study_tsvs/SEQ_Open-R3_imaging_study.tsv
/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/imaging_study_tsvs/SEQ_Open-R3_imaging_study.tsv has 22957 records.

Output written to file: /Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/imaging_study_tsvs/SEQ_Open-A3_imaging_study.tsv
/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/imaging_study_tsvs/SEQ_Open-A3_imaging_study.tsv has 9824 records.

Output written to file: /Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/imaging_study_tsvs/SEQ_NCU-R4_imaging_study.tsv
/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/imaging_study_tsvs/SEQ_NCU-R4_imaging_study.tsv has 0 records.
Removing empty file: /Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/imaging_study_tsvs/SEQ_NCU-R4_imaging_study.tsv

Output 

In [11]:
dupes = vt.loc[vt.duplicated(subset='submitter_id',keep=False)].sort_values(by='submitter_id')
print("There are {} duplicate submitter_ids in the list of imaging studies.".format(len(dupes)))
if len(dupes) > 0:
    display(dupes)



There are 0 duplicate submitter_ids in the list of imaging studies.


## Reformat the imaging_study data to prepare for mapping
---
* **NaN values**: In the mapping table, a `study_description` or `study_modality` with a value of `null` or `NaN` is listed as `[blank]`.
* **Case-insensitivity**: we should ignore capitalization of `study_modality` and `study_description` strings when we do the look-up. We'll use `str.casefold()` to ignore case for matching.
* **CR/DX Modality**: in the mapping table, all versions of "CR" and "DX" modalities are denoted "CR, DX"; so, we need to change all occurrences of other spellings to match that exactly.
* **Duplicated study UIDs**: Identify any duplicated study UIDs and remove them from studies.


In [12]:
## Make a copy of the master imaging_study table with only data relevant to LOINC mapping, and change NaN to "[blank]"
studies = copy.deepcopy(vt[['submitter_id','study_description','study_modality']])
#studies.index = studies['submitter_id']
studies['study_description'].fillna("[blank]", inplace=True)
studies['study_modality'].fillna("[blank]", inplace=True)

## Make the mapping case-insensitive
studies['study_description'] = studies['study_description'].str.casefold()
studies['study_modality'] = studies['study_modality'].str.casefold()

## remove any leading/trailing whitespaces
studies['study_description'] = studies['study_description'].str.strip()
studies['study_modality'] = studies['study_modality'].str.strip()


## Fix any CR/DX study_modality to all be "cr, dx" to match mapping table
studies.replace({"study_modality":{
    'cr':'cr, dx',
    'dx':'cr, dx',
    'cr,dx':'cr, dx',
    'dx,cr':'cr, dx',
    'dx, cr':'cr, dx',
    'crdx':'cr, dx',
    'dxcr':'cr, dx'}},
    inplace=True)

## Fix any CT/PT study_modality to all be "pt, ct" to match mapping table
studies.replace({"study_modality":{
    'pt':'pt, ctpt',
    'ctpt':'pt, ctpt',
    'ptct':'pt, ctpt',
    'ct, pt':'pt, ctpt',
    'pt, ct':'pt, ctpt',
    'pt,ct':'pt, ctpt',
    'ct,pt':'pt, ctpt'}},
    inplace=True)

## Change any "<NONE>" values for study_description to "[blank]" to match the mapping table
studies.replace({"study_description":{
    '<none>':'[blank]'}},
    inplace=True)

## Identify and drop duplicates
dupes = studies.loc[studies.duplicated(keep=False)]
studies = studies.drop_duplicates()
display(studies)



,submitter_id,study_description,study_modality
0,1.2.826.0.1.3680043.10.474.232451.56322,xr port chest 1v,"cr, dx"
1,1.2.826.0.1.3680043.10.474.232451.71121,xr port chest 1v,"cr, dx"
2,1.2.826.0.1.3680043.10.474.639127.3478,chest 2 views (routine),"cr, dx"
3,1.2.826.0.1.3680043.10.474.514382.568905,"xr chest 1 vw, frontal","cr, dx"
4,1.2.826.0.1.3680043.10.474.302028.3459938,xr chest portable 1 view,"cr, dx"
...,...,...,...
32776,2.16.840.1.114274.1818.52698089236797955732278...,chest pa & lateral (rad)-cs,"cr, dx"
32777,2.16.840.1.114274.1818.56100095492960449271516...,ct abdomen w contrast (routine),ct
32778,2.16.840.1.114274.1818.50984339427924945982056...,xr chest 1 vw portable,"cr, dx"
32779,2.16.840.1.114274.1818.53227214212647572403557...,ct chest abdomen pelvis w contrast (routine),ct


In [13]:
## Check modalities for presence in the mapping table and frequency in MIDRC data:
study_modalities = list(set(studies.study_modality))
print("There are the following values of study_modality in the MIDRC imaging_study data: \n{}".format(study_modalities))

missing_modalities = set(study_modalities).difference(modalities)

mfreq = {}
for mod in missing_modalities:
    freq = len(studies.loc[studies['study_modality']==mod])
    mfreq[mod] = freq

print("\nThese modalities are in the study data but not in the mapping table: \n{}".format(mfreq))

There are the following values of study_modality in the MIDRC imaging_study data: 
['pt, ctpt', 'mr', 'ct', 'cr, dx', 'rf']

These modalities are in the study data but not in the mapping table: 
{}


## Do mapping using pandas pd.merge(): 
---

1) Merge `loinc_code` in `mapping` dataframe into the `studies` dataframe on the combination of `study_description` and `study_modality`.

2) Merge the `loinc_long_common_name`, `loinc_method`, `loinc_contrast`, and `loinc_system` in `filters` dataframe into the `studies` dataframe on `loinc_code`.



In [14]:
vdf = studies.merge(mapping,on=['study_modality','study_description'],how='left').drop_duplicates()
vdf = vdf.merge(filters,on=['loinc_code','loinc_long_common_name'],how='left')
vdf.drop(columns=['study_description','study_modality'],inplace=True,errors='ignore')
vdf

,submitter_id,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,1.2.826.0.1.3680043.10.474.232451.56322,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
1,1.2.826.0.1.3680043.10.474.232451.71121,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
2,1.2.826.0.1.3680043.10.474.639127.3478,NaN,NaN,NaN,NaN,NaN
3,1.2.826.0.1.3680043.10.474.514382.568905,36554-4,XR Chest Single view,XR,NaN,Chest
4,1.2.826.0.1.3680043.10.474.302028.3459938,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
...,...,...,...,...,...,...
32776,2.16.840.1.114274.1818.52698089236797955732278...,42272-5,XR Chest PA and Lateral,XR,NaN,Chest
32777,2.16.840.1.114274.1818.56100095492960449271516...,NaN,NaN,NaN,NaN,NaN
32778,2.16.840.1.114274.1818.50984339427924945982056...,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
32779,2.16.840.1.114274.1818.53227214212647572403557...,72254-6,CT Chest and Abdomen and Pelvis W contrast IV,CT,W,Chest+Abdomen+Pelvis


In [16]:
## Pull in the original study_modality and study_description (non-lowercase) and rearrange the columns
cols = ['type','project_id','submitter_id','cases.submitter_id','study_modality','study_description','loinc_code','loinc_long_common_name','loinc_method','loinc_contrast','loinc_system']
vdf = vdf.merge(vt[['type','project_id','submitter_id','cases.submitter_id','study_description','study_modality']], on='submitter_id',how='left')[cols]
vdf


,type,project_id,submitter_id,cases.submitter_id,study_modality,study_description,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,imaging_study,SEQ_Open-R3,1.2.826.0.1.3680043.10.474.232451.56322,232451-001153,CR,XR PORT CHEST 1V,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
1,imaging_study,SEQ_Open-R3,1.2.826.0.1.3680043.10.474.232451.71121,232451-002107,CR,XR PORT CHEST 1V,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
2,imaging_study,SEQ_Open-R3,1.2.826.0.1.3680043.10.474.639127.3478,639127-000405,CR,CHEST 2 VIEWS (ROUTINE),NaN,NaN,NaN,NaN,NaN
3,imaging_study,SEQ_Open-R3,1.2.826.0.1.3680043.10.474.514382.568905,514382-006333,DX,"XR CHEST 1 VW, FRONTAL",36554-4,XR Chest Single view,XR,NaN,Chest
4,imaging_study,SEQ_Open-R3,1.2.826.0.1.3680043.10.474.302028.3459938,302028-009025,DX,XR CHEST PORTABLE 1 VIEW,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
...,...,...,...,...,...,...,...,...,...,...,...
32776,imaging_study,SEQ_Open-A3,2.16.840.1.114274.1818.52698089236797955732278...,10008204-uGXkDSf4gEGidZQcRvBtVw,CR,CHEST PA & LATERAL (RAD)-CS,42272-5,XR Chest PA and Lateral,XR,NaN,Chest
32777,imaging_study,SEQ_Open-A3,2.16.840.1.114274.1818.56100095492960449271516...,10000364-2377800,CT,CT ABDOMEN W CONTRAST (ROUTINE),NaN,NaN,NaN,NaN,NaN
32778,imaging_study,SEQ_Open-A3,2.16.840.1.114274.1818.50984339427924945982056...,10022779-8MKoZfrS0Gq7lfoHpRdfw,CR,XR CHEST 1 VW PORTABLE,36589-0,Portable XR Chest AP single view,XR.portable,NaN,Chest
32779,imaging_study,SEQ_Open-A3,2.16.840.1.114274.1818.53227214212647572403557...,10000364-374868,CT,CT CHEST ABDOMEN PELVIS W CONTRAST (ROUTINE),72254-6,CT Chest and Abdomen and Pelvis W contrast IV,CT,W,Chest+Abdomen+Pelvis


In [17]:
mdf = vdf.loc[~vdf['loinc_code'].isna()]
udf = vdf.loc[vdf['loinc_code'].isna()]
fdf = udf.groupby(["study_modality", "study_description"]).size().reset_index(name="Freq").sort_values(by='Freq',ascending=False)
print("Total Studies: {}, Mapped: {}, Unmapped: {}".format(len(sdf),len(mdf),len(udf)))

NameError: name 'sdf' is not defined

In [18]:
## Save results to files
filename="{}/SEQ_LOINC_mapping_results_{}_{}.tsv".format(results_dir,len(vdf),today)
vdf.to_csv(filename, sep='\t', index=False)
print("All LOINC mapping results for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/SEQ_LOINC_mapped_{}_{}.tsv".format(results_dir,len(mdf),today)
mdf.to_csv(filename, sep='\t', index=False)
print("Successfully mapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/SEQ_LOINC_umapped_{}_{}.tsv".format(results_dir,len(udf),today)
udf.to_csv(filename, sep='\t', index=False)
print("Unmapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/SEQ_LOINC_umapped_frequencies_{}_{}.tsv".format(results_dir,len(fdf),today)
fdf.to_csv(filename, sep='\t', index=False)
print("Frequencies of unmapped imaging study description/modality combinations for projects {} saved to file: \n{}\n".format(projects,filename))


All LOINC mapping results for projects ['SEQ_Open-R3', 'SEQ_Open-A3', 'SEQ_NCU-R4', 'SEQ_NCU-A4'] saved to file: 
/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/SEQ_LOINC_mapping_results_32781_2023-5-19.tsv

Successfully mapped imaging studies for projects ['SEQ_Open-R3', 'SEQ_Open-A3', 'SEQ_NCU-R4', 'SEQ_NCU-A4'] saved to file: 
/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/SEQ_LOINC_mapped_32075_2023-5-19.tsv

Unmapped imaging studies for projects ['SEQ_Open-R3', 'SEQ_Open-A3', 'SEQ_NCU-R4', 'SEQ_NCU-A4'] saved to file: 
/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/SEQ_LOINC_umapped_706_2023-5-19.tsv

Frequencies of unmapped imaging study description/modality combinations for projects ['SEQ_Open-R3', 'SEQ_Open-A3', 'SEQ_NCU-R4', 'SEQ_NCU-A4'] saved to file: 
/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/results_2023-5-19/SEQ_LOINC_umapped_frequencies_302_2023-5-19.tsv



In [19]:
## Take a glance at frequency of the unmapped imaging study modality/description combinations
display(fdf)


,study_modality,study_description,Freq
276,DX,XR CHEST LINE PLACEMENT PORTABLE,131
57,CR,XR Portable Chest 1 view,38
278,DX,XR CHEST MORNING ROUTINE PORTABLE PUMP,18
54,CR,XR P PORT ABDOMEN AP 1 VIEW,15
0,CR,0,12
...,...,...,...
142,CT,CT P CHEST WO,1
143,CT,CT PELVIS WO CONTRAST (CYSTOGRAM),1
144,CT,CT PELVIS WO CONTRAST (ROUTINE),1
145,CT,CT PELVIS WWO CONTRAST,1


## Submit the data to staging
---
Use the Gen3SDK function `Gen3Submission.submit_file()` to update the LOINC properties for each of the newly mapped studies in `mdf`.

In [20]:
# Submit the derived data to staging
projects = list(set(mdf['project_id']))
data = {}
for pid in projects:
    print("Submitting data to project '{}'.".format(pid))
    data[pid] = vexp.submit_df(df=mdf.loc[mdf["project_id"]==pid],project_id=pid, chunk_size=1000)


Submitting data to project 'SEQ_Open-A3'.
Submitting ['imaging_study'] DataFrame with 9657 records.
	Chunk 1 (chunk size: 1000, submitted: 0 of 9657)


/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/expansion.py:1972: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(


	 Succeeded: 1000 entities.
	Chunk 2 (chunk size: 1000, submitted: 1000 of 9657)
	 Succeeded: 1000 entities.
	Chunk 3 (chunk size: 1000, submitted: 2000 of 9657)
	 Succeeded: 1000 entities.
	Chunk 4 (chunk size: 1000, submitted: 3000 of 9657)
	 Succeeded: 1000 entities.
	Chunk 5 (chunk size: 1000, submitted: 4000 of 9657)
	 Succeeded: 1000 entities.
	Chunk 6 (chunk size: 1000, submitted: 5000 of 9657)
	 Succeeded: 1000 entities.
	Chunk 7 (chunk size: 1000, submitted: 6000 of 9657)
	 Succeeded: 1000 entities.
	Chunk 8 (chunk size: 1000, submitted: 7000 of 9657)
	 Succeeded: 1000 entities.
	Chunk 9 (chunk size: 1000, submitted: 8000 of 9657)
	 Succeeded: 1000 entities.
	Chunk 10 (chunk size: 1000, submitted: 9000 of 9657)
	 Succeeded: 657 entities.
Finished data submission.
Successful records: 9657
Failed invalid records: 0
Submitting data to project 'SEQ_Open-R3'.
Submitting ['imaging_study'] DataFrame with 22418 records.
	Chunk 1 (chunk size: 1000, submitted: 0 of 22418)


/Users/johnbrightanyaaibe/Documents/Notes/MIDRC/LOINC/expansion.py:1972: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(


	 Succeeded: 1000 entities.
	Chunk 2 (chunk size: 1000, submitted: 1000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 3 (chunk size: 1000, submitted: 2000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 4 (chunk size: 1000, submitted: 3000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 5 (chunk size: 1000, submitted: 4000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 6 (chunk size: 1000, submitted: 5000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 7 (chunk size: 1000, submitted: 6000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 8 (chunk size: 1000, submitted: 7000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 9 (chunk size: 1000, submitted: 8000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 10 (chunk size: 1000, submitted: 9000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 11 (chunk size: 1000, submitted: 10000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 12 (chunk size: 1000, submitted: 11000 of 22418)
	 Succeeded: 1000 entities.
	Chunk 13 (chunk size: 1000, submitted: 12000 of 22418)
	 Succeed